# Economic Connectedness

Partial replication on two studies carried out on social capital. The studies appeared in the journal Nature as you can see below:

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital I: measurement and associations with economic mobility. Nature 608, 108–121 (2022). [https://doi.org/10.1038/s41586-022-04996-4](https://doi.org/10.1038/s41586-022-04996-4).

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital II: determinants of economic connectedness. Nature 608, 122–134 (2022). [https://doi.org/10.1038/s41586-022-04997-3](https://doi.org/10.1038/s41586-022-04997-3).


### Geography of Social Capital in the United States

Replication of an interactive [Figure 2a of the first paper](https://www.nature.com/articles/s41586-022-04996-4/figures/2).

In the figure, the social capital is represented by the Economic Connectedness (EC). Economic Connectedness is the degree to which people with low and high Socioeconomic Status (SES) are friends with each other. More formally, to define EC we start by measuring each individual's $i$ share of friends from SES quantile $Q$:

$$ f_{Q, i} = \frac{[\textrm{Number of friends in SES quantile}\ Q]_i}{\textrm{Total number of friends of}\ i} $$

Then we normalize $f_{Q,i}$ by the share of individuals in the sample who belong to quantile $Q$, $w_Q$ (for example, $w_Q = 0.1$ for deciles) to get the Individual Economic Connectedness (IEC):

$$ \mathrm{IEC}_{Q, i} = \frac{f_{Q, i}}{w_Q} $$

The level of EC in a community $c$ is the defined as the mean level of individual EC of low-SES $L$ (for example, below-median) members of that community, as follows:

$$ \mathrm{EC}_{c} = \frac{\sum_{i \in L \cap c}\mathrm{IEC}_i}{N_{Lc}} $$

where $N_{Lc}$ is the number of low-SES individuals in community $c$.

In the figure and in what follows, the EC is twice the share of friends with above-median SES among people with below-median SES; that follows from the above definitions for $w_Q = 0.5$.

The map is constructed using Plotly. The data are displayed per county. When hovering a county, it should display the name of the county and the state it belongs to, the code of the county, and the Economic Connectedness of the county. If there are no data for a particular county, it should be painted with gold, and the economic connectedness should be given as "NA".

Social capital data source: [Social Capital Atlas Datasets](https://data.humdata.org/dataset/social-capital-atlas).

In [ ]:
import pandas as pd
from urllib.request import urlopen
import json
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_county = pd.read_csv("data/social_capital_county.csv")

In [ ]:
# Here we load a GeoJSON file containing the geometry information for US counties, where feature.id is a FIPS (Federal Information Processing Standards) code.
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# Transforming GeoJSON file with its counties into a dataframe in order to be merged into the main county dataframe (so we fill up our dataframe with the missing counties).
df_counties = pd.DataFrame.from_records(counties['features'], columns=['id'] )
df_counties.rename(columns={'id': 'county'}, inplace=True)
df_counties.county = df_counties.county.apply(lambda x: str(x).zfill(5))
data_county.county = data_county.county.apply(lambda x: str(x).zfill(5))
data_counties_processed = pd.merge(data_county, df_counties, on='county', how='outer')

In [ ]:
# Filling NA values
data_counties_processed.ec_county.fillna(value=0, inplace=True)
data_counties_processed.county_name.fillna(value='NA', inplace=True)
data_counties_processed['economic_connectedness'] = data_counties_processed.ec_county.apply(lambda x : round(x, 2))
data_counties_processed['economic_connectedness'] = data_counties_processed.economic_connectedness.apply(lambda x: 'NA' if x==0 else x)

In [ ]:
# Categorizing economic_connectedness
data_counties_processed['color_space'] = pd.cut(data_counties_processed['ec_county'], [-1, 0, 0.58, 0.67, 0.72, 0.76, 0.81, 0.85, 0.90, 0.97, 1.06, float('inf')])
data_counties_processed['color_space'] = data_counties_processed['color_space'].astype(str)

In [ ]:
# Setting up custom counties and staes through Federal Communications Commission

fips_counties = pd.read_csv("https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt", delimiter = "\t")
fips_counties = fips_counties.rename(columns= {'Federal Information Processing System (FIPS) Codes for States and Counties':'county'})

#######
bi_fips = fips_counties['county'].apply(lambda x: str(x).lstrip()[0:5].isdigit())
custom_counties = fips_counties[bi_fips]

custom_counties['county'] = custom_counties['county'].apply(lambda x: x.lstrip())
custom_counties = pd.DataFrame(custom_counties.county.str.split(' ',1).tolist(),
                                 columns = ['fips','county'])
custom_counties.county = custom_counties.county.apply(lambda x: x.replace("County", "").strip())

custom_counties.rename(columns= {'fips':'county', 'county':'county_name'}, inplace=True)

#######################

si_fips = fips_counties['county'].apply(lambda x: x.lstrip()[0:2].isdigit() and not x.lstrip()[0:3].isdigit())
custom_states = fips_counties[si_fips]

custom_states['county'] = custom_states['county'].apply(lambda x: x.lstrip())
custom_states = pd.DataFrame(custom_states.county.str.split(' ',1).tolist(),
                                  columns = ['fips','state'])


In [ ]:
# Interactive map creation
fig = px.choropleth(
                    data_counties_processed,
                    geojson=counties,
                    locations='county',
                    color='color_space',
                    color_discrete_map={
                        '(-1.0, 0.0]': 'gold',
                        '(0.0, 0.58]': 'red',
                        '(0.58, 0.67]': 'orangered' ,
                        '(0.67, 0.72]': 'lightsalmon' ,
                        '(0.72, 0.76]': 'mistyrose',
                        '(0.76, 0.81]': 'whitesmoke',
                        '(0.81, 0.85]': 'lightblue',
                        '(0.85, 0.9]': 'skyblue',
                        '(0.9, 0.97]': 'cornflowerblue',
                        '(0.97, 1.06]': 'royalblue' ,
                        '(1.06, inf]': 'darkblue',
                   },
                   scope="usa",
                   hover_name = 'county_name',
                   hover_data={
                       'county': True,
                       'economic_connectedness': True
                   },
                   labels={
                       'economic_connectedness': 'Economic Connectedness',
                       'county': 'county',
                       'color_space': 'Economic Connectedness'
                   },
                   category_orders={
                       'color_space': ['(1.06, inf]','(0.97, 1.06]','(0.9, 0.97]','(0.85, 0.9]', '(0.81, 0.85]','(0.76, 0.81]','(0.72, 0.76]','(0.67, 0.72]', '(0.58, 0.67]','(0.0, 0.58]','(-1.0, 0.0]']
                   }
                  )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

### Economic Connectedness and Outcomes

Replication of [Figure 4 of the first paper](nature.com/articles/s41586-022-04996-4/figures/4). The figure is a scatter plot of upward income mobility against economic connectedness (EC) for the 200 most populous US counties. The income mobility is obtained from the [Opportunity Atlas](https://www.nber.org/papers/w25147), whose replication data can be found [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NKCQM1).

In [ ]:
# Reading data from Opportunitly Atlas source file in order to obtain income mobility.
county_outcomes = pd.read_csv("data/county_outcomes_simple.csv")

In [ ]:
# Selecting from counties data frame the 200 most populous.
top_200_pop_index = data_county['pop2018'].nlargest(200).index
top_200_counties_pop = data_county.iloc[top_200_pop_index]
top_200_counties_pop = top_200_counties_pop[['county', 'pop2018', 'ec_county']]

In [ ]:
# Selecting and formatting income mobility from county outcomes source file in order to be merged with the 200 most populous counties.
county_outcomes_processed = county_outcomes[['state', 'county', 'kfr_pooled_pooled_p25']]
county_outcomes_processed.state = county_outcomes_processed.state.apply(lambda x: str(x).zfill(2))
county_outcomes_processed.county = county_outcomes_processed.county.apply(lambda x: str(x).zfill(3))
county_outcomes_processed['county'] = county_outcomes_processed.state + county_outcomes_processed.county
county_outcomes_processed = county_outcomes_processed.drop(['state'], axis=1)

In [ ]:
# Merging are two tables into one in order to create the lmplot.
ec_outcomes = pd.merge(top_200_counties_pop, county_outcomes_processed, on='county', how='left')
ec_outcomes = pd.merge(ec_outcomes, custom_counties, on='county', how='left')

In [ ]:
# Storing cities that they are going to be annotated. (Minneapolis and Indianapolis belong to Hennepin and Marion Counties respectively)
ec_outcomes_sel_cities= ['New York', 'Salt Lake', 'Hennepin', 'San Francisco', 'Marion']
pointed_cities = ec_outcomes.loc[ec_outcomes.county_name.isin(ec_outcomes_sel_cities)]

In [ ]:
# Apply the default theme
sns.set_theme()

ax = sns.lmplot(x="ec_county", y="kfr_pooled_pooled_p25", data=ec_outcomes, height=5, aspect=1.5)

# Setting labels
ax.set(xlabel='Economic Connectedness', ylabel='Predicted Household Income Rank \nfor Children with Parents at 25th Income Percentile')

# New York arrow annotation
plt.annotate('New York', (pointed_cities.ec_county.loc[pointed_cities.county_name=='New York'], pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='New York']),( pointed_cities.ec_county.loc[pointed_cities.county_name=='New York'] , pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='New York'] + 0.08), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='black'), size=10, ha="center")

# Indianapolis arrow annotation
plt.annotate('Indianapolis', (pointed_cities.ec_county.loc[pointed_cities.county=='18097'], pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county=='18097']),( pointed_cities.ec_county.loc[pointed_cities.county=='18097'] + 0.18, pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county=='18097'] -0.013), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='black'), size=10, ha="center")

# Salt Lake City arrow annotation
plt.annotate('Salt Lake City', (pointed_cities.ec_county.loc[pointed_cities.county_name=='Salt Lake'], pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='Salt Lake']),( pointed_cities.ec_county.loc[pointed_cities.county_name=='Salt Lake'] , pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='Salt Lake'] + 0.06), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='black'), size=10, ha="center")

# San Francisco arrow annotation
plt.annotate('San Francisco', (pointed_cities.ec_county.loc[pointed_cities.county_name=='San Francisco'], pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='San Francisco']),( pointed_cities.ec_county.loc[pointed_cities.county_name=='San Francisco'] , pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='San Francisco'] - 0.08), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='black'), size=10, ha="center")

# Minneapolis arrow annotation
plt.annotate('Minneapolis', (pointed_cities.ec_county.loc[pointed_cities.county_name=='Hennepin'], pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='Hennepin']),( pointed_cities.ec_county.loc[pointed_cities.county_name=='Hennepin'] + 0.12 , pointed_cities.kfr_pooled_pooled_p25.loc[pointed_cities.county_name=='Hennepin'] - 0.06), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='black'), size=10, ha="center")

plt.show()

### Upward Income Mobility, Economic Connectedness, and Median House Income

Replication of [Figure 6 of the first paper](https://www.nature.com/articles/s41586-022-04996-4/figures/6). The figure is a scatter plot of economic connectedness (EC) against median household income. The color of the dots corresponds to the child's income rank in adulthood given that the parents' income is in the 25th percentile. The colors correspond to five intervals, which are the quintiles dividing our data.

Used data from replication package of the papers with data from the Social Capital Atlas Datasets.

In [ ]:
# Reading social capital data by zip code.
data_zip = pd.read_csv("data/social_capital_zip.csv")
zip_covariates = pd.read_stata("data/zip_covariates.dta", columns=['zip', 'kfr_pooled_pooled_p25','med_inc_2018'])

In [ ]:
# Selecting and formatting zip and economic connectedness from our dataset.
ec_zip = data_zip[['zip','ec_zip']]
ec_zip.zip = ec_zip.zip.apply(lambda x: str(x).zfill(5))

In [ ]:
# Formatting zip code in order to be ready for merge
zip_covariates.zip = zip_covariates.zip.apply(lambda x: str(x).zfill(5))

In [ ]:
# Merging tables from replication package and social capital
merged_zips = pd.merge(ec_zip, zip_covariates, on='zip', how='left')

# Dropping NA values
merged_zips = merged_zips.dropna()

# Formating median income column
merged_zips.med_inc_2018 = merged_zips.med_inc_2018.apply(lambda x: int(x))

# Keeping data above 30.000 and belove 100.000 median income
merged_zips = merged_zips.loc[merged_zips.med_inc_2018 <= 100000]
merged_zips = merged_zips.loc[merged_zips.med_inc_2018 >= 30000]

In [ ]:
# Creating Upward Mobility column in order to bin kfr_pooled_pooled_p25 intervals
merged_zips['Upward Mobility'] = pd.cut(merged_zips['kfr_pooled_pooled_p25'], [0,0.38,0.41,0.44,0.48,5])

In [ ]:
sns.set_style('dark')

sns.set(rc={'figure.figsize':(12, 7)})

palette = ["#FD1C03", "#FFAE42", "#FFF8DC", "#3BB9FF", "#00008B"]

g = sns.scatterplot(x='med_inc_2018', y='ec_zip', data=merged_zips, hue='Upward Mobility',
                    palette=sns.color_palette(palette, 5))

handles, labels = g.get_legend_handles_labels()
a = g.legend(reversed(handles), reversed(labels) , title = "Upward Mobility", loc="lower right")

g.set(xlabel='Median Household Income in ZIP Code (US$)', ylabel='Economic Connectedness')

a.texts[0].set_text(" >48")
a.texts[1].set_text("44 - 48")
a.texts[2].set_text("41 - 44")
a.texts[3].set_text("38 - 41")
a.texts[4].set_text(" <38")
plt.show()

### Friending Bias and Exposure by High School

Replication of [Figure 5a of the second paper](https://www.nature.com/articles/s41586-022-04997-3/figures/5). The figure depicts the share of students with high parental Socioeconomic Status (SES) against the friending bias of students of low parental SES, with data from the Social Capital Atlas Datasets.

Note that in order to get the share of high-parental-SES students, which is the $x$-axis, we took the mean exposure to high-parental-SES individuals by high school and divide it by two. That is because the mean exposure to high-parental-SES individuals by high schools is defined as two times the average share of high parental-SES individuals within three birth cohorts, averaged over low-parental-SES users.

Note also that both $x$ and $y$ axis are percentages and the $y$ axis is reversed.


In [ ]:
# Reading data for social capital high schools
data_high_school = pd.read_csv("data/social_capital_high_school.csv")

In [ ]:
# Mean exposure to high parental SES individuals and divide it by two
data_high_school['exposure_parent_ses_hs'] = data_high_school['exposure_parent_ses_hs'].apply(lambda x: (x/2)*100)
data_high_school['bias_parent_ses_hs'] = data_high_school['bias_parent_ses_hs'].apply(lambda x: x*100)

In [ ]:
# Setting up school to be annotated
annotated_high_schools = ['00941729', '060474000432', '170993000942',
                          '170993001185', '170993003989', '171449001804',
                          '250327000436', '360009101928', '370297001285',
                          '483702004138', '250843001336', '062271003230',
                          '010237000962', '00846981', '00852124']
#
friend_bias_expo = data_high_school[['exposure_parent_ses_hs','bias_parent_ses_hs']].dropna()

# Getting cells with specific high school id
annotations = data_high_school.loc[data_high_school['high_school'].isin(annotated_high_schools)]
annotations = annotations[['exposure_parent_ses_hs','bias_parent_ses_hs', 'high_school', 'high_school_name']]

In [ ]:
# Function to create annotation. It takes as inputs a dataframe, pyplot instance, and x, y text coords.
def create_annotation(df, plt, x_text_cord, y_text_cord):
    plt.annotate(df.high_school_name.iloc[0], (df.exposure_parent_ses_hs.iloc[0], df.bias_parent_ses_hs.iloc[0]),( df.exposure_parent_ses_hs.iloc[0] + x_text_cord, df.bias_parent_ses_hs.iloc[0] - y_text_cord), bbox=dict(facecolor='white',alpha=0.8), arrowprops=dict(arrowstyle="-|>", connectionstyle="arc3", lw=1 , color='cyan'), size=10, ha="center")


In [ ]:
a = sns.scatterplot(data=friend_bias_expo, x="exposure_parent_ses_hs", y="bias_parent_ses_hs", alpha=0.4, color='black', linewidth=0)

# Reversing y axis
a.invert_yaxis()

phillips_exeter_academy = annotations.loc[annotations.high_school_name=='Phillips Exeter Academy']
bishop_gorman_hs = annotations.loc[annotations.high_school_name=='Bishop Gorman HS']
dalton_school = annotations.loc[annotations.high_school_name=='Dalton School']
john_l_magnet_school = annotations.loc[annotations.high_school_name=='John L Leflore Magnet School']
berkeley_hs = annotations.loc[annotations.high_school_name=='Berkeley HS']
north_hollywood = annotations.loc[annotations.high_school_name=='North Hollywood Sr HS']
lane_technical = annotations.loc[annotations.high_school_name=='Lane Technical HS']
lincoln_park = annotations.loc[annotations.high_school_name=='Lincoln Park HS']
payton_college = annotations.loc[annotations.high_school_name=='Payton College Preparatory HS']
evanston_twp = annotations.loc[annotations.high_school_name=='Evanston Twp HS']
cambridge_rindge = annotations.loc[annotations.high_school_name=='Cambridge Rindge And Latin']
new_bedford = annotations.loc[annotations.high_school_name=='New Bedford HS']
brooklyn_technical = annotations.loc[annotations.high_school_name=='Brooklyn Technical HS']
west_charlotte = annotations.loc[annotations.high_school_name=='West Charlotte HS']
lake_highlands = annotations.loc[annotations.high_school_name=='Lake Highlands HS']

a.set(xlabel='Share of high-parental-SES students (%)', ylabel='Friending bias among low-parental-SES students (%)')

create_annotation(phillips_exeter_academy, plt, 0.5, 5 )
sns.scatterplot(data= phillips_exeter_academy, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(bishop_gorman_hs, plt, 4.2, -4.3 )
sns.scatterplot(data= bishop_gorman_hs, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(dalton_school, plt, -0.2, 4)
sns.scatterplot(data= dalton_school, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(john_l_magnet_school, plt, 2.2, -3.3)
sns.scatterplot(data= john_l_magnet_school, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(berkeley_hs, plt, -0.2, 1.3)
sns.scatterplot(data= berkeley_hs, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(north_hollywood, plt, -13.2, -3.3)
sns.scatterplot(data= north_hollywood, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(lane_technical, plt, 5.2, 4.6)
sns.scatterplot(data= lane_technical, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(lincoln_park, plt, 1.2, -9)
sns.scatterplot(data= lincoln_park, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(payton_college, plt, 0, 12.1)
sns.scatterplot(data= payton_college, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(evanston_twp, plt, 7, -1)
sns.scatterplot(data= evanston_twp, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(cambridge_rindge, plt, 17, -1)
sns.scatterplot(data= cambridge_rindge, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(new_bedford, plt, -2, -4)
sns.scatterplot(data= new_bedford, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(brooklyn_technical, plt, -4, 3)
sns.scatterplot(data= brooklyn_technical, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(west_charlotte, plt, -0.51, 4)
sns.scatterplot(data= west_charlotte, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

create_annotation(lake_highlands, plt, 4.51, 2)
sns.scatterplot(data= lake_highlands, x= 'exposure_parent_ses_hs', y= 'bias_parent_ses_hs', color="cyan", linewidth=0)

plt.show()

### Friending Bias vs. Racial Diversity

Replication of [Extended Data Figure 3](https://www.nature.com/articles/s41586-022-04997-3/figures/9) of the second paper. The figure depicts friending bias against racial diversity. Racial diversity is defined by the [Herfindahl-Hirschman Index (HHI)](https://en.wikipedia.org/wiki/Herfindahl%E2%80%93Hirschman_index), borrowed from investing. Translated here, it is $ 1−\sum_{i}{s_i}^2$, where $s_i$ is the fraction of race/ethnicity $i$ (Black, White, Asian, Hispanic, Native American).

The figure contains two scatter plots with their respective regression lines, one for college data and the other for neighborhood data. Each of the two plots displays binned data (that's why you don't see loads of dots and diamonds). The bins are produced by dividing the $x$-axis into ventiles (i.e., 5 percentile point bins); then we plot the mean of the $y$-axis variable against the appropriate mean of the $x$-axis variable in each ventile.

The mean of the $x$-axis variable, the HHI index, is the weighted mean of HHI:

* For the college plot, the weights are given by the mean number of students per cohort.

* For the neighborhood plot, the weights are given by the number of children with below-national-median parental household income.

The $y$-axis variable:

* For the college plot, it is the mean of the college friending bias.

* For the neighborhood plot, it is the mean of the neighborhood friending bias.


In [ ]:
# Reading social capital, zip covariates and college characteristics data
social_capital_college = pd.read_csv("data/social_capital_college.csv")
social_capital_zip = pd.read_csv("data/social_capital_zip.csv")
college_characteristics = pd.read_stata("data/college_characteristics.dta")
zip_covariates = pd.read_stata("data/zip_covariates.dta")
# social_capital_zip = pd.read_csv("Q5_files/scz.csv")

In [ ]:
# Formating social capital df and zip covariates df in order to be merged
social_capital_zip['zip'] = social_capital_zip['zip'].apply(lambda x: str(x).zfill(5))
zip_covariates['zip'] = zip_covariates['zip'].apply(lambda x: str(x).zfill(5))
zip_characteristics_merged = pd.merge(social_capital_zip, zip_covariates, on=['zip', 'num_below_p50'], how='left')

In [ ]:
# Initializing racial diversity
zip_characteristics_merged['hhi'] = zip_characteristics_merged.apply(lambda x: 1 - (x.share_white_2018 ** 2 + x.share_black_2018 ** 2+ x.share_natam_2018 ** 2 + x.share_asian_2018 ** 2 +  x.share_hispanic_2018 ** 2), axis=1)
zip_characteristics_merged['weighted_hhi'] = zip_characteristics_merged.num_below_p50 * zip_characteristics_merged.hhi
zip_characteristics_merged = zip_characteristics_merged[['zip', 'num_below_p50', 'nbhd_bias_zip', 'hhi', 'weighted_hhi']]

In [ ]:
# Creating binned data
zip_characteristics_merged['percentile_5'] = pd.qcut(zip_characteristics_merged.hhi,20)
zip_characteristics_merged['weighted_hhi_mean_zip'] = zip_characteristics_merged.groupby('percentile_5')['hhi'].transform('mean')
zip_characteristics_merged['bias_mean_zip']= zip_characteristics_merged.groupby('percentile_5')['nbhd_bias_zip'].transform('mean')
zip_binned_data = zip_characteristics_merged[['bias_mean_zip','weighted_hhi_mean_zip','percentile_5']].drop_duplicates()

In [ ]:
social_capital_college.college.max()
college_characteristics.college.max()
#out 7 digits for both

In [ ]:
# Formatting social capital college and college characteristics
# Maximum integer digits of college id is 7.
social_capital_college.college = social_capital_college.college.apply(lambda x: str(x).zfill(7))
college_characteristics.college = college_characteristics.college.apply(lambda x: str(x).zfill(7))
merged_college_characteristics = pd.merge(social_capital_college, college_characteristics, on=['college', 'mean_students_per_cohort'], how='left')

In [ ]:
# Initializing racial diversity
frac_black = merged_college_characteristics.black_share_fall_2000
frac_hispanic = merged_college_characteristics.hisp_share_fall_2000
frac_asian = merged_college_characteristics.asian_or_pacific_share_fall_2000
frac_white = 1 - frac_black - frac_hispanic - frac_asian
merged_college_characteristics['hhi'] = 1 - (frac_white ** 2 + frac_black ** 2 + frac_asian ** 2 + frac_hispanic **2)
merged_college_characteristics['weighted_spc'] = merged_college_characteristics.mean_students_per_cohort * merged_college_characteristics.hhi
merged_college_characteristics = merged_college_characteristics[['college','bias_own_ses_college', 'mean_students_per_cohort', 'hhi', 'weighted_spc']]

In [ ]:
# Creating binned data
merged_college_characteristics['percentile_5'] = pd.qcut(merged_college_characteristics.hhi,20)
merged_college_characteristics['weighted_hhi_mean_col'] = merged_college_characteristics.groupby('percentile_5')['hhi'].transform('mean')
merged_college_characteristics['bias_mean_col']= merged_college_characteristics.groupby('percentile_5')['bias_own_ses_college'].transform('mean')
col_binned_data = merged_college_characteristics[['bias_mean_col','weighted_hhi_mean_col','percentile_5']].drop_duplicates()

In [ ]:
# Merging college and zip binned data
binned_data = pd.merge(col_binned_data, zip_binned_data, how='outer', on='percentile_5')

In [ ]:
# Apply the default theme
sns.set_theme()
# draw regplot
sns.regplot(x = "weighted_hhi_mean_zip",
            y = "bias_mean_zip",
            data = binned_data,
            scatter_kws={"color": "darkorange"}, line_kws={"color": "darkorange"},
            dropna = True)

# draw regplot
ax = sns.regplot(x = "weighted_hhi_mean_col",
            y = "bias_mean_col",
            data = binned_data,
            marker="D",
            scatter_kws={"color": "cornflowerblue"}, line_kws={"color": "cornflowerblue"},
            dropna = True
            )


ax.set_xlabel('Racial Diversity (Herfindahl-Hirschman Index) in Group',fontsize=9)
ax.set_ylabel('Friending Bias among Low-Ses Individuals (%)',fontsize=9)

# show the plot
plt.show()